# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Thu 2023-01-05 21:27:49.793539 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE
Thu 2023-01-05 21:27:49.794095 [INFO ] - [TRANSPORT] Attempting to bind to 0.0.0.0:0
Thu 2023-01-05 21:27:49.794183 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Thu 2023-01-05 21:27:49.795521 [INFO ] - [TRANSPORT] Attempting to bind to 0.0.0.0:62013
Thu 2023-01-05 21:27:49.819019 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7fbcfa677330>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        print(f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        print(f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            print(f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
client_tiface

server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
server_tiface

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

(<FilterInterface at 0x7fbcfa9eafc0>, <FilterInterface at 0x7fbcfa9ead40>)

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command_response', 'get_notifications', 'notif_poll_ms', 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient3168", client_version: "0.0.666" }

In [11]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient3168", client_version: "0.0.666" } }

In [12]:
await client_fiface.command_response(filter_cmd)

Thu 2023-01-05 21:28:09.553528 [TRACE] - [FILTER] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient3168", client_version: "0.0.666" } }
LTI(client)124 skipped 123: received transport_cmd NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 250ms }
Thu 2023-01-05 21:28:09.554851 [TRACE] - [TRANSPORT] Filter Request: NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 250ms }
LTI(client)125: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI(client)126: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient3168", client_version: "0.0.666" } }] }
Thu 2023-01-05 21:28:09.555313 [TRACE] - [FILTER] Transport Command Accepted
Thu 2023-01-05 21:28:09.555468 [TR

Accepted

LTI(client)130: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:28:09.654453 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)131: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)132: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: None, action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-05 21:28:09.750668 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: None, action: KeepAlive { latest_response_ack: 1 } }] }
LTI(client)133: got transport_rsp back from Transport layer: T

Thu 2023-01-05 21:28:10.078383 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:55041).
Thu 2023-01-05 21:28:10.078421 [TRACE] - [FILTER] Took packet: Request { sequence: 4, response_ack: Some(3), cookie: None, action: KeepAlive { latest_response_ack: 3 } }
LTI(server)290 skipped 6: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:55041), tid: ProcessUniqueId { prefix: 1, offset: 0 } }
Thu 2023-01-05 21:28:10.078446 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:55041).
Thu 2023-01-05 21:28:10.078454 [TRACE] - [FILTER] Took packet: Request { sequence: 5, response_ack: Some(3), cookie: None, action: KeepAlive { latest_response_ack: 3 } }
Thu 2023-01-05 21:28:10.079370 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:55041), tid: ProcessUniqueId { prefix: 1, offset: 0 } }
LTI(server)291: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:28:10.079832 [TRACE] - [FILTER] Tran

LTI(server)310: got transport notifications: []
LTI(client)155: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 6, request_ack: Some(6), code: OK } }]
Thu 2023-01-05 21:28:10.357722 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:10.357740 [TRACE] - [FILTER] Took packet: Response { sequence: 6, request_ack: Some(6), code: OK }
LTI(client)156: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 6 }, retry_interval: 250ms }], packets: [Request { sequence: 7, response_ack: Some(5), cookie: None, action: KeepAlive { latest_response_ack: 5 } }] }
Thu 2023-01-05 21:28:10.358013 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: 

...

LTI(client)1241: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 208, request_ack: Some(208), code: OK } }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Thu 2023-01-05 21:28:37.928675 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:37.928720 [TRACE] - [FILTER] Took packet: Response { sequence: 208, request_ack: Some(208), code: OK }
LTI(client)1242: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 107 }, retry_interval: 250ms }], packets: [Request { sequence: 210, response_ack: Some(209), cookie: None, action: KeepAlive { latest_response_ack: 209 } }] }
Thu 2023-01-05 21:28:37.930282 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:6201

LTI(client)1259: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 213, request_ack: Some(213), code: OK } }]
Thu 2023-01-05 21:28:38.334214 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:38.334240 [TRACE] - [FILTER] Took packet: Response { sequence: 213, request_ack: Some(213), code: OK }
LTI(client)1260: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 128 } }
Thu 2023-01-05 21:28:38.334764 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 128 } }
LTI(client)1261: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:28:38.334944 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)1262: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:620

LTI(client)1273: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 214, request_ack: Some(214), code: OK } }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Thu 2023-01-05 21:28:38.738453 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:38.738479 [TRACE] - [FILTER] Took packet: Response { sequence: 214, request_ack: Some(214), code: OK }
LTI(client)1274: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 110 }, retry_interval: 250ms }], packets: [Request { sequence: 216, response_ack: Some(215), cookie: None, action: KeepAlive { latest_response_ack: 215 } }] }
Thu 2023-01-05 21:28:38.739926 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:6201

LTI(client)1291: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 219, request_ack: Some(219), code: OK } }]
Thu 2023-01-05 21:28:39.142526 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:39.142549 [TRACE] - [FILTER] Took packet: Response { sequence: 219, request_ack: Some(219), code: OK }
LTI(client)1292: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 113 } }
Thu 2023-01-05 21:28:39.143175 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 113 } }
LTI(client)1293: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:28:39.143332 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)1294: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:620

LTI(client)1305: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 220, request_ack: Some(220), code: OK } }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Thu 2023-01-05 21:28:39.546882 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:39.546904 [TRACE] - [FILTER] Took packet: Response { sequence: 220, request_ack: Some(220), code: OK }
LTI(client)1306: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 133 }, retry_interval: 250ms }], packets: [Request { sequence: 222, response_ack: Some(221), cookie: None, action: KeepAlive { latest_response_ack: 221 } }] }
Thu 2023-01-05 21:28:39.547472 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:6201

In [13]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

LTI(client)1323: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 225, request_ack: Some(225), code: OK } }]
Thu 2023-01-05 21:28:39.952603 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:39.952636 [TRACE] - [FILTER] Took packet: Response { sequence: 225, request_ack: Some(225), code: OK }
LTI(client)1324: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 118 } }
Thu 2023-01-05 21:28:39.953366 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 118 } }
LTI(client)1325: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:28:39.953730 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)1326: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:620

NewResponseCode { endpoint: Endpoint(127.0.0.1:62013), code: OK }

LTI(client)1329: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 224, request_ack: Some(224), code: OK } }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 226, request_ack: Some(226), code: OK } }]
Thu 2023-01-05 21:28:40.155602 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:40.155631 [TRACE] - [FILTER] Took packet: Response { sequence: 224, request_ack: Some(224), code: OK }
Thu 2023-01-05 21:28:40.155640 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:40.155643 [TRACE] - [FILTER] Took packet: Response { sequence: 226, request_ack: Some(226), code: OK }
LTI(client)1330: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqu

LTI(client)1347: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)1348: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 141 }, retry_interval: 250ms }], packets: [Request { sequence: 230, response_ack: Some(229), cookie: None, action: KeepAlive { latest_response_ack: 229 } }] }
Thu 2023-01-05 21:28:40.560677 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 141 }, retry_interval: 250ms }], packets: [Request { sequence: 230, response_ack: Some(229), cookie: None, action: KeepAlive { latest_response_ack: 229 } }] }
LTI(client)1349: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:28:40.560770 [TRACE] - [TRANSPORT] UDP Codec data: Request { sequence: 230, response_ack: Some(229), c

LTI(client)1361: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 230, request_ack: Some(230), code: OK } }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 232, request_ack: Some(232), code: OK } }]
Thu 2023-01-05 21:28:40.966176 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:40.966195 [TRACE] - [FILTER] Took packet: Response { sequence: 230, request_ack: Some(230), code: OK }
Thu 2023-01-05 21:28:40.966207 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:28:40.966214 [TRACE] - [FILTER] Took packet: Response { sequence: 232, request_ack: Some(232), code: OK }
LTI(client)1362: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqu

...

LTI(client)4219: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 769, request_ack: Some(769), code: OK } }]
Thu 2023-01-05 21:29:52.913462 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:52.913485 [TRACE] - [FILTER] Took packet: Response { sequence: 769, request_ack: Some(769), code: OK }
LTI(client)4220: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 430 } }
Thu 2023-01-05 21:29:52.914199 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 430 } }
LTI(client)4221: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:29:52.914514 [TRACE] - [FILTER] Transport Command Accepted
Thu 2023-01-05 21:29:53.000670 [TRACE] - [TRANSPORT] UDP Codec data: Response { sequence: 768, request_ack: Som

LTI(client)4235: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 772, request_ack: Some(772), code: OK } }]
LTI(client)4236: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 432 }, retry_interval: 250ms }], packets: [Request { sequence: 773, response_ack: Some(771), cookie: None, action: KeepAlive { latest_response_ack: 771 } }] }
Thu 2023-01-05 21:29:53.318868 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:53.318909 [TRACE] - [FILTER] Took packet: Response { sequence: 772, request_ack: Some(772), code: OK }
Thu 2023-01-05 21:29:53.320867 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, o

LTI(client)4249: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)4250: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 375 }, retry_interval: 250ms }], packets: [Request { sequence: 775, response_ack: Some(774), cookie: None, action: KeepAlive { latest_response_ack: 774 } }] }
Thu 2023-01-05 21:29:53.724877 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 375 }, retry_interval: 250ms }], packets: [Request { sequence: 775, response_ack: Some(774), cookie: None, action: KeepAlive { latest_response_ack: 774 } }] }
Thu 2023-01-05 21:29:53.725031 [TRACE] - [TRANSPORT] UDP Codec data: Request { sequence: 775, response_ack: Some(774), cookie: None, action: KeepAlive { latest_response_ack: 774 } }
LTI(client)4251: got tr

LTI(client)4265: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 777, request_ack: Some(778), code: OK } }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 778, request_ack: Some(778), code: OK } }]
Thu 2023-01-05 21:29:54.128424 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:54.128446 [TRACE] - [FILTER] Took packet: Response { sequence: 777, request_ack: Some(778), code: OK }
Thu 2023-01-05 21:29:54.128459 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:54.128464 [TRACE] - [FILTER] Took packet: Response { sequence: 778, request_ack: Some(778), code: OK }
LTI(client)4266: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 377 } }
Thu 2023-01-05 21:29:54.128935 [TRACE] - [TRANSPORT] Filter Request: Dr

LTI(client)4281: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 779, request_ack: Some(779), code: OK } }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Thu 2023-01-05 21:29:54.532558 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:54.532582 [TRACE] - [FILTER] Took packet: Response { sequence: 779, request_ack: Some(779), code: OK }
LTI(client)4282: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 384 }, retry_interval: 250ms }], packets: [Request { sequence: 781, response_ack: Some(780), cookie: None, action: KeepAlive { latest_response_ack: 780 } }] }
Thu 2023-01-05 21:29:54.533164 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:6201

In [14]:
cookie = new_response_code.response_code.cookie
cookie

LTI(client)4299: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 784, request_ack: Some(784), code: OK } }]
Thu 2023-01-05 21:29:54.939915 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:54.939939 [TRACE] - [FILTER] Took packet: Response { sequence: 784, request_ack: Some(784), code: OK }
LTI(client)4300: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 3, offset: 381 } }
Thu 2023-01-05 21:29:54.940656 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 3, offset: 381 } }
LTI(client)4301: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:29:54.940949 [TRACE] - [FILTER] Transport Command Accepted
Thu 2023-01-05 21:29:55.011612 [TRACE] - [TRANSPORT] UDP Codec data: Response { sequence: 783, request_ack: Som

LTI(client)4315: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 787, request_ack: Some(787), code: OK } }]
Thu 2023-01-05 21:29:55.346767 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
LTI(client)4316: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 366 }, retry_interval: 250ms }], packets: [Request { sequence: 788, response_ack: Some(786), cookie: None, action: KeepAlive { latest_response_ack: 786 } }] }
Thu 2023-01-05 21:29:55.346785 [TRACE] - [FILTER] Took packet: Response { sequence: 787, request_ack: Some(787), code: OK }
Thu 2023-01-05 21:29:55.347377 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, o

LTI(client)4335: got transport_rsp back from Transport layer: TransportRsp::AcceptedThu 2023-01-05 21:29:55.752910 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 390 }, retry_interval: 250ms }], packets: [Request { sequence: 791, response_ack: Some(790), cookie: None, action: KeepAlive { latest_response_ack: 790 } }] }
Thu 2023-01-05 21:29:55.753116 [TRACE] - [TRANSPORT] UDP Codec data: Request { sequence: 791, response_ack: Some(790), cookie: None, action: KeepAlive { latest_response_ack: 790 } }

Thu 2023-01-05 21:29:55.754810 [TRACE] - [FILTER] Transport Command Accepted
Thu 2023-01-05 21:29:55.770764 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:55041).
Thu 2023-01-05 21:29:55.770782 [TRACE] - [FILTER] Took packet: Request { sequence: 791, response_ack: Some(790), cookie: None, action: KeepAlive { latest_response_ack: 790 } }
LTI(server)6831 skipped

LTI(client)4349: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 793, request_ack: Some(794), code: OK } }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 794, request_ack: Some(794), code: OK } }]
Thu 2023-01-05 21:29:56.159106 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:56.159143 [TRACE] - [FILTER] Took packet: Response { sequence: 793, request_ack: Some(794), code: OK }
Thu 2023-01-05 21:29:56.159171 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:56.159177 [TRACE] - [FILTER] Took packet: Response { sequence: 794, request_ack: Some(794), code: OK }
LTI(client)4350: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 372 } }
Thu 2023-01-05 21:29:56.159766 [TRACE] - [TRANSPORT] Filter Request: Dr

LTI(client)4365: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 795, request_ack: Some(795), code: OK } }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Thu 2023-01-05 21:29:56.565797 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:56.565822 [TRACE] - [FILTER] Took packet: Response { sequence: 795, request_ack: Some(795), code: OK }
LTI(client)4366: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 377 }, retry_interval: 250ms }], packets: [Request { sequence: 797, response_ack: Some(796), cookie: None, action: KeepAlive { latest_response_ack: 796 } }] }
Thu 2023-01-05 21:29:56.566316 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:6201

LTI(client)4383: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 800, request_ack: Some(800), code: OK } }]
Thu 2023-01-05 21:29:56.970096 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:56.970117 [TRACE] - [FILTER] Took packet: Response { sequence: 800, request_ack: Some(800), code: OK }
LTI(client)4384: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 382 } }
Thu 2023-01-05 21:29:56.970617 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 382 } }
LTI(client)4385: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-05 21:29:56.970912 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)4386: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:620

LTI(client)4395: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 802, request_ack: Some(802), code: OK } }]
LTI(client)4396: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 385 }, retry_interval: 250ms }], packets: [Request { sequence: 803, response_ack: Some(801), cookie: None, action: KeepAlive { latest_response_ack: 801 } }] }
Thu 2023-01-05 21:29:57.377310 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Thu 2023-01-05 21:29:57.377346 [TRACE] - [FILTER] Took packet: Response { sequence: 802, request_ack: Some(802), code: OK }
Thu 2023-01-05 21:29:57.377787 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, o

LTI(client)4413: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Thu 2023-01-05 21:29:57.880991 [TRACE] - [TRANSPORT] UDP Codec data: Response { sequence: 804, request_ack: Some(805), code: OK }
LTI(client)4414: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 804, request_ack: Some(805), code: OK } }]
Thu 2023-01-05 21:29:57.910959 [TRACE] - [TRANSPORT] UDP Codec data: Response { sequence: 805, request_ack: Some(805), code: OK }
LTI(client)4415: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 805, request_ack: Some(805), code: OK } }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)4416: got transport notifications: []
Thu 2023-01-05 21:29:58.131001 [TRACE] - [TRANSPORT

In [15]:
cookie

In [ ]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
server_notifications[0]

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [ ]:
client_fiface.get_notifications()